<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

This notebook is modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
%%capture --no-stderr
# %pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import autogen
import os

# config_list_gpt4 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
#     },
# )

config_list_gpt4 = [
    {
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_API_KEY"),
        # "api_key": str(os.environ["OPENAI_API_KEY"]),
    },
    {
        "model": "gpt-4-32k",
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
]


# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

## Start Chat

In [ ]:
Marketing_Expert = """
Name: Alex the Marketing Expert
Age: 32
Background: Holds a Master's degree in Marketing and has a passion for cycling. Alex has previously worked with several sports brands and has a deep understanding of the biking community.
Personality Traits: Creative, data-driven, and trend-savvy. Excels in digital marketing strategies.
Goals: To increase brand visibility and engage more with the biking community through innovative marketing campaigns.
Challenges: Keeping up with rapidly changing marketing trends and consumer preferences.
Digital Marketing: Expert in SEO, social media advertising, and email marketing campaigns specifically tailored for the sports industry.
Brand Development: Skilled in developing and maintaining a strong brand identity that resonates with the biking community.
Market Research: Proficient in conducting market analysis to identify new trends and customer needs in the biking market.
Content Creation: Talented in creating engaging content (blogs, videos, social media posts) to drive brand awareness and customer engagement.
"""
Sales_Expert = """
Name: Emma the Sales Expert
Age: 28
Background: Emma has a Bachelor's degree in Business Administration and has been working in sales for 5 years, specializing in sports equipment. She's known for her exceptional customer service skills and product knowledge.
Personality Traits: Outgoing, persuasive, and empathetic. Great at building relationships and understanding customer needs.
Goals: To consistently exceed sales targets and develop strong, long-lasting relationships with key clients.
Customer Relationship Management: Excel at building and maintaining strong relationships with clients, ensuring long-term customer loyalty.
Product Knowledge: In-depth understanding of biking products and the ability to articulate product benefits effectively to customers.
Sales Strategy: Proficient in developing and implementing effective sales strategies to target various customer segments in the biking market.
Negotiation Skills: Strong negotiation skills to close deals and secure new business opportunities.
"""
Manager = """
Name: Michael the Manager
Age: 40
Background: With an MBA and over 15 years of experience in management roles, Michael has a solid track record in leading teams and driving company growth. He's particularly adept at strategic planning and operations management.
Personality Traits: Leadership-oriented, analytical, and decisive. Excellent at problem-solving and team management.
Goals: To streamline operations for efficiency, foster a positive work culture, and drive the company towards its strategic goals.
Strategic Planning: Excellent at setting strategic goals for the company and developing plans to achieve these goals.
Team Management: Skilled in managing diverse teams, fostering a collaborative and productive work environment.
Financial Acumen: Strong understanding of budgeting, financial planning, and resource allocation to maximize efficiency and profitability.
Operational Oversight: Proficient in overseeing daily operations, ensuring processes are streamlined and goals are met.
"""
Business_Overview = """
The market for sports articles for bikers is dynamic and multifaceted, catering to a wide range of customers from casual riders to professional athletes. It encompasses various segments including road biking, mountain biking, BMX, and leisure biking. The demand is driven by a growing interest in outdoor activities, fitness, and eco-friendly modes of transportation.
Key Segments:
Road Biking: This segment includes high-performance bikes and gear for speed and endurance, appealing to serious athletes and fitness enthusiasts. The market is dominated by a few major brands such as Trek, Cannondale, and Specialized.
Internal Weakness: Limited Online Presence and Digital Marketing Expertise.
Your company currently has a minimal online presence, with a basic website and limited activity on social media platforms. The in-house team lacks expertise in digital marketing, SEO, and e-commerce strategies. This deficiency results in reduced online visibility and a lack of engagement with potential customers, especially those who predominantly shop and seek information online.
Market Opportunity: Growing Demand for E-Bikes.
"""
email = """
Subject: Strategy Meeting Invitation: Capitalizing on E-Bike Market Opportunity

Body:

Dear Alex, Emma, and Michael,

I hope this email finds you well. As we navigate through an increasingly competitive market, it is essential that we continually align our strategies with emerging opportunities and internal capabilities. To this end, I would like to invite you to a strategic meeting to discuss a significant opportunity in the e-bike market and address our current internal challenges that are im
"""
meeting_opener = """
"Good morning everyone,

Thank you for joining this crucial meeting today. I'm excited to discuss an opportunity that has the potential to significantly impact our market positioning and overall growth.

Firstly, let's talk about the opportunity at hand. We're witnessing a substantial shift in consumer preferences towards e-bikes, especially among urban commuters. This trend isn't just a fleeting one; it's driven by increasing environmental awareness, the need for efficient commuting options, an aging population, and a growing interest in fitness. In fact, the global e-bike market is expected to grow at a CAGR of 9.01% from 2021 to 2028, reaching a value of $38.6 billion by 2028.
"""

In [ ]:
llm_config = {"config_list": config_list_gpt4}
marketing = autogen.AssistantAgent(
    name="Alex_the_Marketing_Expert",
    system_message=Marketing_Expert,
    # human_input_mode="TERMINATE",
    llm_config=llm_config,
)
sales = autogen.UserProxyAgent(
    name="Emma_the_Sales_Expert",
    # human_input_mode="NEVER",
    human_input_mode="TERMINATE",
    system_message=Sales_Expert,
    llm_config=llm_config,
)
# manager = autogen.AssistantAgent(
#     name="Michael_the_Manager",
#     system_message=Manager,
#     # human_input_mode="TERMINATE",
#     llm_config=llm_config,
# )

In [31]:
from memgpt.autogen.memgpt_agent import (
    create_autogen_memgpt_agent,
    create_memgpt_autogen_agent_from_config,
)
from memgpt.presets.presets import DEFAULT_PRESET

DEBUG = True

interface_kwargs = {
    "debug": DEBUG,
    "show_inner_thoughts": DEBUG,
    "show_function_outputs": DEBUG,
}

config_list_memgpt = [
    {
        # "model": "gpt-4-1106-preview",  # gpt-4-turbo (https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo)
        "model": "gpt-4-1106-preview",  # gpt-4-turbo (https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo)
        "preset": DEFAULT_PRESET,
        "model_wrapper": None,
        "api_type": "open_ai",
        "model_endpoint_type": "openai",
        "model_endpoint": "https://v1",
        "context_window": 128000,  # gpt-4-turbo
    },
]

llm_config_memgpt = {"config_list": config_list_memgpt}

manager = create_memgpt_autogen_agent_from_config(
    "Michael_the_Mannager",
    llm_config=llm_config_memgpt,
    nonmemgpt_llm_config=llm_config,
    system_message=Manager,
    interface_kwargs=interface_kwargs,
    human_input_mode="NEVER",  # Set a default auto-reply message here (non-empty auto-reply is required for LM Studio)
)

KeyError: 'model_endpoint'

In [32]:
groupchat = autogen.GroupChat(
    agents=[marketing, sales, manager], messages=[], max_round=5
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
sales.initiate_chat(
    manager,
    message=Business_Overview + email + meeting_opener,
)
# type exit to terminate the chat